<div style="margin-top: 25px;">
    <h1 style="width: 100%; text-align: center; font-size: 5rem;">
        Web scraper
    </h1>
    <div style="text-align: center; margin-top:15px;">
        <h4 style="margin: 5px;">Rodrigo C. Barros</h4>
        <h4 style="margin: 5px;">Nathan S. Gavenski</h4>
        <h4 style="margin: 5px;">Escola Politécnica - PUCRS</h4>
    </div>
    <div style="margin-top: 35px;">
        <p>
            Este notebook irá entrar na página do IMDB e buscar pelo filme de escolha do aluno. <br>
            Para que a execução seja feita corretamente o <strong>driver do Chrome</strong> (baixado do Moodle) deve ser colocado na pasta <strong>"root"</strong> do projeto. <br>
            Como os usuários da PUCRS não são administradores das máquinas não iremos criar ambientes de desenvolvimento, então para que as dependências corretas estejam instaladas é preciso executar a primeira célula.
        </p>
    </div>
</div>

<h5>Essa célula irá instalar as dependências</h5>

In [ ]:
!pip install -r requirements.txt

<h5>Vamos agora importar a classe criado pelo professor e o webdriver do Selenium</h5>

In [ ]:
from Movie import Movie
from selenium import webdriver

<h5> Na próxima célula iremos abrir o navegador via o selenium e escolher o filme que queremos pesquisar </h5>
<p> Note que caso não tenha colocado o Driver no "root" do projeto o caminho deve mudar </p>

In [ ]:
driver = webdriver.Chrome(executable_path='./chromedriver.exe')
driver.maximize_window()

driver.get(url="https://www.imdb.com/")
title = input('Qual filme gostaria de pesquisa? ')
print(f'Título escolhido: {title}')

<h5> Agora vamos pesquisa pelo título que colocamos na célula anterior </h5>
<p> Note que o site do IMDB limita o resultado da busca para os 5 primeiros títulos. <br> 
    Se o título que vocês escolheu tem mais de 5 filmes não iremos selecionar eles agora! </p>

In [ ]:
# Procura pelo WebElement com id = 'navbar-query' e escreve o título
search = driver.find_element_by_id('navbar-query')
search.send_keys(title)

# Procura pelo botão de busca e clica nele
driver.find_element_by_id('navbar-submit-button').click()

# Procura pela tabela de filmes e grava todos os filmes listados
table = driver.find_element_by_class_name('findList')
rows = table.find_elements_by_class_name('findResult')
print(f'Número de filmes encontrados: {len(rows)}')

<h5> Com a lista de filmes em tela e já salvas na variável "rows", vamos agora buscar todos os links </h5>
<p> Já que não estamos testando o site do IMDB não precisamos nos preocupar em usar o site como um usuário <br>Podemos salvar os links de cada filme em uma lista e depois só navegar para cada URL buscando as informações que queremos.</p>

In [ ]:
# Aqui vamos iterar sobre a lista "rows" e pegar o WebElement de cada filme
movies = [r for r in rows if title in r.text]
print(f'Número de filmes que tenham o título "{title}": {len(movies)}')

# Com cada filme separado em lista, podemos então buscar o link para não precisar navegar no site
links = [link.find_element_by_tag_name('a').get_attribute('href') for link in movies]

print('\nLink de cada filme:')
for link in links:
    print('\t' + link)

<h5> Com os links em mão agora só precisamos direcionar o WebDriver para as URLs encontradas e salvar as informações na classe Movie </h5>

In [ ]:
movies = []

for link in links:
    driver.get(link)
    try:
        # Aqui estamos usando xpath para selecionar o elemento h1 de div's que tenham a classe "title_wrapper"    
        title = driver.find_element_by_xpath('//div[@class="title_wrapper"]/h1').text

        # Depois vamos selecionar o elemento com a classe "poster" e nele achar a tag img e pegar o atributo src
        poster_url = driver.find_element_by_class_name('poster')\
                                .find_element_by_tag_name('img')\
                                .get_attribute('src')

        # Usar o xpath para encontrar o elemento span contendo atributo "itemprop" com valor "ratingValue"
        rate = driver.find_element_by_xpath('//span[@itemprop="ratingValue"]').text

        # Aqui vamos achar o elemento que tenha o id = "titleStoryLine"
        storyline = driver.find_element_by_id('titleStoryLine')      
        plot_keywords = storyline.find_elements_by_class_name('itemprop')

        # Com os elementos pai já salvos agora podemos pegar a descrição, palavras chave e gêneros de cada filme
        description = storyline.find_element_by_xpath('//div/p/span').text
        keywords = [key.text for key in plot_keywords]    
        genres = [link.text for link in storyline\
                        .find_element_by_xpath('//h4[contains(text(), "Genres")]')\
                        .find_element_by_xpath('./..')\
                        .find_elements_by_tag_name('a')]

        # Agora só precisamos adicionar o novo filme em nossa lista de filmes
        movies.append(Movie(title, poster_url, rate, description, keywords, genres))
    except:         
        continue

In [ ]:
from PIL import Image
print(f'Filme: {movies[0].title}')
print(f'Avaliação: {movies[0].rate}')
print(f'Palavras Chaves: {movies[0].keywords}')
print(f'Gênero: {movies[0].genres}')
print(f'Descrição: {movies[0].description}')
print('Poster: ')
display(Image.open(movies[0].poster_path))

# Desafio! 

#### Agora que já entendemos como o framework funciona podemos tentar melhorar nosso script
* Mudar o script para pegar todos os filmes (além dos 5 primeiros)
* Tentar pegar informações de box office de cada filme
* Tentar baixar o trailer de cada filme